<a href="https://colab.research.google.com/github/z-guard/analysis/blob/main/notebooks/geocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ジオコーディング

In [ ]:
!pip install -q geocoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DIR_NAME = '/content/drive/MyDrive/z-gard/data'

In [ ]:
import os
import requests
import json
import pandas as pd
from urllib.parse import urlparse
import geocoder
import math
import numpy as np

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [ ]:
df_pop_master = pd.read_csv(os.path.join(DIR_NAME, 'population_master.csv'), dtype='str')[['town_id', '市区町村名', '町名', '町丁目']]
print(df_pop_master.shape)

(3145, 4)


In [ ]:
df_pop_master.head()

,town_id,市区町村名,町名,町丁目
0,101003,千代田区,丸の内,丸の内１丁目
1,101004,千代田区,丸の内,丸の内２丁目
2,101005,千代田区,丸の内,丸の内３丁目
3,101007,千代田区,大手町,大手町１丁目
4,101008,千代田区,大手町,大手町２丁目


In [ ]:
def number_to_kanji(str):
    return str.translate(str.maketrans({'１':'一', '２':'二', '３':'三', '４':'四', '５':'五', '６':'六', '７':'七', '８':'八', '９':'九'}))

# 緯度経度取得
def add_lat_lon(df_master):
    lats = []
    lons = []
    for i, row in df_master.iterrows():
        if (i % 500) == 0:
            print(f'{i}')
        chocho = number_to_kanji(row['町丁目'])
        ret = geocoder.osm('東京都' + row['市区町村名'] + chocho, timeout=10)
        if not ret.ok:
            print(i, row['市区町村名'], chocho)
        lats.append(ret.lat)
        lons.append(ret.lng)
    df_copy = df_master.copy()
    df_copy['緯度'] = lats
    df_copy['経度'] = lons
    return df_copy


# 距離計算
def get_distance(lat1, lon1, lat2, lon2):
    _lat1 = lat1 * math.pi / 180
    _lon1 = lon1 * math.pi / 180
    _lat2 = lat2 * math.pi / 180
    _lon2 = lon2 * math.pi / 180
    _tmp = math.cos(_lat1) * math.cos(_lat2) * math.cos(_lon2 - _lon1) + math.sin(_lat1) * math.sin(_lat2)
    _tmp = 1 if _tmp > 1 else -1 if _tmp < -1 else _tmp
    return 6371 * math.acos(_tmp)

In [ ]:
%%time
df_pop_lat_lon = add_lat_lon(df_pop_master)

0
500
1000
1050 品川区 水面
1354 大田区 羽田沖水面
1355 大田区 多摩川河川敷（上流）
1356 大田区 多摩川河川敷（下流）
1357 大田区 ふるさとの浜辺公園
1500
1532 世田谷区 深沢七丁目
2000
2500
2829 葛飾区 お花茶屋一丁目
2830 葛飾区 お花茶屋二丁目
2831 葛飾区 お花茶屋三丁目
3000
CPU times: user 45.6 s, sys: 2.81 s, total: 48.4 s
Wall time: 26min 8s


In [ ]:
place_photo_file = os.path.join(DIR_NAME, 'place_photo.csv')
if os.path.exists(place_photo_file):
    df_place = pd.read_csv(place_photo_file, dtype={'no': 'str'})[['no', 'lat', 'lng']]
    print(df_place.shape)

    df_pop_place = pd.concat([
        df_pop_lat_lon.set_index('town_id'),
        df_place.set_index('no')
    ], axis=1)
    df_pop_place.index.name = 'town_id'
    print(df_pop_place.shape)

    df_pop_place['dist'] = df_pop_place.apply(lambda x: get_distance(x['緯度'], x['経度'], x['lat'], x['lng']), axis=1)
    df_pop_place.loc[(df_pop_place['dist'] > 0.8)|(df_pop_place['緯度'].isna()), '緯度'] = df_pop_place['lat']
    df_pop_place.loc[(df_pop_place['dist'] > 0.8)|(df_pop_place['経度'].isna()), '経度'] = df_pop_place['lng']
    df_pop_lat_lon = df_pop_place.drop(columns=['lat', 'lng', 'dist']).reset_index()
    print(df_pop_lat_lon.shape)

(2809, 3)
(3145, 7)
(3145, 6)


In [ ]:
# df_pop_lat_lon = df_pop_lat_lon.rename(columns={'地域ID':'town_id'})

In [ ]:
df_pop_lat_lon[(df_pop_lat_lon['緯度'].isna())|(df_pop_lat_lon['経度'].isna())]

,town_id,市区町村名,町名,町丁目,緯度,経度
1050,109158,品川区,水面,水面,NaN,NaN
1354,111269,大田区,羽田沖水面,羽田沖水面,NaN,NaN
1357,111273,大田区,ふるさとの浜辺公園,ふるさとの浜辺公園,NaN,NaN
2830,122050,葛飾区,お花茶屋,お花茶屋２丁目,NaN,NaN


## 近隣の町

In [ ]:
lat_min = df_pop_lat_lon['緯度'].min()
lat_max = df_pop_lat_lon['緯度'].max()
lon_min = df_pop_lat_lon['経度'].min()
lon_max = df_pop_lat_lon['経度'].max()
div_count = 30
lat_div = (lat_max - lat_min) / (div_count - 1)
lon_div = (lon_max - lon_min) / (div_count - 1)

In [ ]:
# 隣の町までの距離
get_distance(lat_min, lon_min, (lat_min+lat_div), (lon_min+lon_div))

1.5198371124248922

In [ ]:
df_pop_lat_lon['lat_index'] = round((df_pop_lat_lon['緯度'] - lat_min) / lat_div)
df_pop_lat_lon['lon_index'] = round((df_pop_lat_lon['経度'] - lon_min) / lon_div)
df_pop_lat_lon['map_index'] = df_pop_lat_lon['lat_index'] * div_count + df_pop_lat_lon['lon_index']
df_pop_lat_lon['map_index'] = df_pop_lat_lon['map_index'].fillna(-1).astype(int)
print(df_pop_lat_lon['map_index'].nunique())

563


In [ ]:
# nearby search用の緯度経度
nearby_search = []
for map_i in df_pop_lat_lon['map_index'].unique():
    if map_i < 0:
        continue
    lat_index = map_i // div_count
    lon_index = map_i % div_count
    lat = lat_min + lat_div * (lat_index + 0.5)
    lon = lon_min + lon_div * (lon_index + 0.5)
    nearby_search.append({'map_index': map_i, 'lat': lat, 'lon': lon})

df_nearby_search = pd.DataFrame(nearby_search).sort_values('map_index')
print(df_nearby_search.shape)

df_nearby_search.to_csv(os.path.join(DIR_NAME, 'nearby_search_base.csv'), index=False)

(562, 3)


In [ ]:
def get_near_map_index(map_index):
    if (map_index < 0) or (map_index >= div_count*div_count):
        return []
    # 自分の周囲8個分のmap_indexを返す
    ret = []
    lat_index = map_index // div_count
    lon_index = map_index % div_count
    for lat in [-1, 0, 1]:
        for lon in [-1, 0, 1]:
            lat_tmp = lat_index + lat
            lon_tmp = lon_index + lon
            if (lat_tmp < 0) or (lat_tmp >= div_count) or (lon_tmp < 0) or (lon_tmp >= div_count):
                continue
            ret.append(lat_tmp * div_count + lon_tmp)
    return ret

In [ ]:
df_map_all = pd.DataFrame(range(div_count*div_count), columns=['map_index'])
df_map_all = df_map_all.set_index('map_index')

map_table_org = df_pop_lat_lon.groupby('map_index')['town_id'].apply(list)
map_table = pd.concat([map_table_org, df_map_all], axis=1).sort_index().fillna(0)
print(map_table.shape)

(901, 1)


In [ ]:
df_map_index = map_table_org.reset_index()
df_map_index['near_list'] = df_map_index['map_index'].apply(lambda x: get_near_map_index(x))
df_map_index = df_map_index[df_map_index['town_id'].notna()].reset_index(drop=True)
print(len(df_map_index))

563


In [ ]:
all_town_list = []
for i, row in df_map_index.iterrows():
    town_list = []
    for map_id in row['near_list']:
        tmp_l = map_table.loc[map_id]['town_id']
        if tmp_l != 0:
            town_list += tmp_l
    all_town_list.append(town_list)

df_map_index['near_town_list'] = all_town_list

In [ ]:
df_town_all = pd.concat([
    df_pop_lat_lon.set_index('town_id'),
    df_map_index.drop(columns=['map_index', 'near_list']).explode('town_id').set_index('town_id')
], axis=1)
print(df_town_all.shape)

(3145, 9)


In [ ]:
df_town_all.head()

,市区町村名,町名,町丁目,緯度,経度,lat_index,lon_index,map_index,near_town_list
town_id,,,,,,,,,
101003,千代田区,丸の内,丸の内１丁目,35.681560,139.767201,15.0,17.0,467,"[101005, 101010, 101011, 101013, 101014, 10104..."
101004,千代田区,丸の内,丸の内２丁目,35.680022,139.763447,15.0,16.0,466,"[101016, 101017, 101018, 101020, 101005, 10101..."
101005,千代田区,丸の内,丸の内３丁目,35.676952,139.763476,14.0,16.0,436,"[103023, 103024, 103025, 103044, 103045, 10304..."
101007,千代田区,大手町,大手町１丁目,35.688069,139.763929,15.0,16.0,466,"[101016, 101017, 101018, 101020, 101005, 10101..."
101008,千代田区,大手町,大手町２丁目,35.686301,139.768087,15.0,17.0,467,"[101005, 101010, 101011, 101013, 101014, 10104..."


In [ ]:
neighbor_town_l = []
for i, row in df_town_all.iterrows():
    lat1 = row['緯度']
    lon1 = row['経度']
    near_l = row['near_town_list']
    town_l = []
    for near_id in near_l:
        if near_id == i:
            # 自分自身は除外
            continue
        near_town = df_town_all.loc[near_id]
        lat2 = near_town['緯度']
        lon2 = near_town['経度']
        dist = get_distance(lat1, lon1, lat2, lon2)
        # 半径1km以内の町
        if dist <= 1:
            town_l.append({'town_id':near_id, 'distance':round(dist, 2)})
    if len(town_l) == 0:
        print(i, row['町丁目'], len(town_l))
    neighbor_town_l.append(town_l)

109158 水面 0
111200 羽田空港２丁目 0
111201 羽田空港３丁目 0
111269 羽田沖水面 0
111273 ふるさとの浜辺公園 0
122050 お花茶屋２丁目 0


In [ ]:
def get_nearest_town(x):
    if len(x) == 0:
        return []
    _tmp = pd.DataFrame(x).sort_values('distance').reset_index(drop=True)
    if len(_tmp) > 5:
       _tmp = _tmp.iloc[:5]
    return list(_tmp['town_id'])

In [ ]:
df_town_all['neighbor_town'] = neighbor_town_l
df_town_all['neighbor_town'] = df_town_all['neighbor_town'].apply(get_nearest_town)

In [ ]:
df_town_all = df_town_all.drop(columns=['lat_index', 'lon_index', 'near_town_list'])
df_town_all.to_csv(os.path.join(DIR_NAME, 'geocoding.csv'), encoding='utf-8_sig', index=True)

In [ ]:
df_town_all.head()

,市区町村名,町名,町丁目,緯度,経度,map_index,neighbor_town
town_id,,,,,,,
101003,千代田区,丸の内,丸の内１丁目,35.681560,139.767201,467,"[102090, 101004, 102002, 101008, 102046]"
101004,千代田区,丸の内,丸の内２丁目,35.680022,139.763447,466,"[101005, 101003, 101040, 102002, 101013]"
101005,千代田区,丸の内,丸の内３丁目,35.676952,139.763476,436,"[101004, 101014, 101013, 102008, 102002]"
101007,千代田区,大手町,大手町１丁目,35.688069,139.763929,466,"[101095, 101086, 101008, 101087, 101096]"
101008,千代田区,大手町,大手町２丁目,35.686301,139.768087,467,"[102048, 102047, 102049, 102046, 101007]"
